This R "notebook" contains a complete (albeit highly simplified, and probably not very good for planning purposes) four-step model for the central Research Triangle region (Orange, Durham, and Wake counties), North Carolina.

First, we need to run some R code to set up the modeling system. This will likely take several minutes.

In [ ]:
system("git clone https://github.com/mattwigway/rdu-four-step-model.git")
setwd("rdu-four-step-model")
source("setup.R")

Next, we need to load the model itself.

In [ ]:
model = load_model("rdu.model")

The model requires a land use scenario as well. We load that here. The land use scenario is just an Excel sheet with the total number of households of different types (e.g. levels of vehicle ownership, income) in each Census tract, the number of jobs in each Census tract, and the locations of the Census tracts.

In many four-step models, a custom set of Transportation Analysis Zones (TAZs) is used; in this model, our TAZs are just Census tracts.

I have provided several land use scenarios:
    - `baseline.xlsx` - This is the baseline (current conditions)
    - `projected.xlsx` - This is the 2030 projected scenario, based roughly on our demographic projections from a few weeks back
    - `compact.xlsx` - This is a 2030 projected scenario where growth is concentrated in downtown Durham, Raleigh, and Chapel Hill

By changing the R code below, you can change what scenario is loaded.

In [ ]:
landuse = load_landuse_scenario("baseline.xlsx")

## Trip generation

The first step of the four-step model is trip generation. Here, we generate the productions (number of trips taken by households in each tract) and attractions (number of trips that go to a particular tract). There are different models by trip type (Home-Based Work, Home-Based Other, and Non-Home-Based) and time period (AM Peak, midday, PM Peak, overnight); to give you an idea of what these models look like, we print out two of them before we run the trip generation function.

### Production model, AM Peak home-based work trips

In [ ]:
summary(model$production_functions$`AM Peak`$HBW)

### Attraction model, AM Peak home-based work trips

In [ ]:
summary(model$attraction_functions$`AM Peak`$HBW)

### Running trip generation

Here, we run the trip generation model. This takes a few minutes to run. We save the result in a variable called `trip_ends`, because we have the ends of the trips, but we don't know which productions are associated with which attractions.

In [ ]:
trip_ends = trip_generation(model, landuse)

We can map how many trips are generated in different locations, by trip type and time of day.

In [ ]:
map_trip_generation(trip_ends$productions, "AM Peak", "HBW")

Since there are no other other features on this map, it's a bit hard to read, but by comparing with other maps of the region you can clearly see that the highest densities of trips are generated by the three major cities: Raleigh, Durham, and Chapel Hill, with lower densities generated by suburban areas and the lowest densities by rural areas.

We can similarly map attractions:

In [ ]:
map_trip_generation(trip_ends$attractions, "AM Peak", "HBW")

As one might expect, the trip attractions for work trips are more concentrated---there are fewer "hotspots," but they attract many more trips per square kilometer—downtown Raleigh in particular, but UNC's campus is also visible.

## Trip distribution

After the trip generation step, we have the number of trips produces in each zone and attracted to each zone, but we don't know which trips from zone A go to zone B. That is the job of the trip distribution step. In this model, a gravity function is used to distribute trips; trips are more likely to go to destinations with higher attractiveness, but less likely to go to destinations far away. How far away is controlled by the power in the gravity function. Since people may be willing to travel different amounts for different purposes, we estimate separate powers for each of our three trip types. The powers are shown below:

In [ ]:
model$distribution_betas

Now, we can run the trip distribution step. The result is a table of how many trips of each type occur between each pair of tracts at each time period.

In [ ]:
flows = trip_distribution(model, landuse, trip_ends)

The trip distribution step is harder to map, because we have a number of trips for every _pair_ of tracts. We'll map the trips originating in Orange County tract 107.05, which is where our Plan 720 Towers apartment project is going to be located in Carrboro (shown in blue). Feel free to experiment with changing the origin tract to where you live, or anywhere else. You can find the Census tract by searching for an address on [Census Reporter](https://censusreporter.org)

In [ ]:
map_trip_distribution(flows, "AM Peak", "HBW", origin_tract="37135010705")

As expected, many people are expected to travel to workplaces that are local. But we also see significant travel to downtown Raleigh, Duke Hospital, and Research Triangle Park—these are far away but have a lot of attractions.

## Mode choice

We have now forecasted how many trips will flow between each pair of tracts, by type and time period. However, we don't know what mode they will use. This is the function of the mode choice model. This model actually has two mode choice models—one for home-based work/other trips and one for non-home-based trips. The reason for this is that the home-based trip model can use information about the home location to improve the model fit. Here is the mode choice model for non-home-based trips.

In [ ]:
summary(model$mode_choice_models$HB)

Now, we can apply the mode choice model.

In [ ]:
flows_by_mode = mode_choice(model, landuse, flows)

## Network assignemnt/route choice

The last step is to assign the auto trips to the network. Right now, we have forecasted origins, destinations, and modes, but we don't know what roads they use. Often, the two model outputs with the most significant policy implications are congestion levels and emissions. Both require network assignment. Evaluating emissions is beyond the scope of this lesson (if you're interested, check out the [EPA MOVES](https://www.epa.gov/moves) model), but we will model congestion.

In some models, generally in regions where there is much higher transit usage than here in the Research Triangle, there is also a network assignment process for transit.

This model uses an iterative network assignment process known as the Frank-Wolfe algorithm. Running the cell below will assign the forecasted PM Peak trips to the network. This is the most computationally intensive part of the process; it will probably take about 5–10 minutes. As the model runs, it will print out its progress using a metric known as "relative gap"—when it falls below 0.01, the model is finished (this metric is not something you need to remember or know the definition of, but just a tip so you can get a sense of how much longer you have to wait).

In [ ]:
pm_network_flows = network_assignment(model, landuse, flows_by_mode, "PM Peak")

We can map congestion levels based on this data.

In [ ]:
map_congestion(pm_network_flows, model)

This does look like more congestion than I would expect from living here; there are likely inaccuracies in the simple model we're using that would need to be calibrated to match reality if we were actually using it for forecasting.